# OptiGuide Example




In [1]:
# Install Required Packages
%pip install optiguide


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [2]:
openai.api_key_path = "OAI_CONFIG_LIST.txt"

In [4]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)


Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [5]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/yuqingsh/capstone-project-demo/master/farm_lotsizing.py"
response  = requests.get(code_url)
# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from gurobipy import Model, GRB
import pandas as pd
import time


# Calculate statistics
df = pd.read_csv("data/supply_chain_data.csv")
average_demand = df["Number of products sold"].mean()  # average demand per period
demand_std_dev = df["Number of products sold"].std()  # standard deviation of demand
lead_time = 3  # lead time in periods
.
.
.

model.optimize()

print(time.ctime())

# Print the optimal safety stock
if m.status == GRB.OPTIMAL:
    print(f"Optimal Safety Stock: {safety_stock.x}")
else:
    print("No optimal solution found")



In [6]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to rely on just one seed supplier for Farm 2?
Answer Code:
```python
z = model.addVars(suppliers, vtype=GRB.BINARY, name="z")
model.addConstr(sum(z[s] for s in suppliers) <= 1, "one_supplier_constraint")
for s in suppliers:
    model.addConstr(x[s, 'farm2'] <= seed_supply_capacity[s] * z[s], "capacity_constraint_farm2")
```

----------
Question: How can we adjust the model if we want to explore a scenario where the planting cost at Farm 1 increases by 10%?
Answer Code:
```python
planting_cost['farm1'] = planting_cost['farm1'] * 1.10
# Update the Objective Function to Reflect New Costs
model.setObjective(
    sum(x[i] * transport_cost_supplier_to_farm[i] for i in transport_cost_supplier_to_farm.keys()) +
    sum(y[j] * transport_cost_farm_to_market[j] for j in transport_cost_farm_to_market.keys()) +
    sum(planting_cost[f] * sum(y[f, m] for m in markets) for f in farms) +
    sum(storage_preservation_cost[m] * market_demand[m] for m in markets),
    GRB.MINIMIZE)

```


"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [7]:
%%capture
agent = OptiGuideAgent(name="OptiGuide Coffee Example",
                  source_code=code,
                   debug_times=1,
                  example_qa="",
                llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    })

user = UserProxyAgent("user", max_consecutive_auto_reply=0,
                         human_input_mode="NEVER", code_execution_config=False)


Now, let's create a user's question.

In [10]:
user.initiate_chat(agent, message="What is the safety stock of the skincare product?")


user (to OptiGuide Coffee Example):

What is the safety stock of the skincare product?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Add the product data
product_data = pd.read_csv("data/skincare_product_data.csv")
average_demand_product = product_data["Number of products sold"].mean()  # average demand per period
demand_std_dev_product = product_data["Number of products sold"].std()  # standard deviation of demand
demand_variability_product = demand_std_dev_product * (
    lead_time**0.5
)  # demand variability over lead time

# Add variable for safety stock of the skincare product
safety_stock_product = model.addVar(vtype=GRB.CONTINUOUS, name="SafetyStockSkincare")

# Set the objective for the skincare product
model.setObjective(safety_stock_product, GRB.MINIMIZE)


In [10]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Update the supply capacity of supplier1
seed_supply_capacity['supplier1'] = 1000 / 2

# Update the constraint for supplier1 in the model
model.remove(m.getConstrByName(f"seed_supply_{suppliers[0]}"))
model.addConstr(sum(x[suppliers[0], f] for f in farms) <= seed_supply_capacity[suppliers[0]], f"seed_supply_{suppliers[0]}")
```

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
# Update the supply capacity of supplier1
seed_supply_capacity['supplier1'] = 1000 / 2

# Update the constraint for supplier1 in the model
